<a href="https://colab.research.google.com/github/hesandism/ML-projects/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['status','msg'])
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['status','msg'])

df_train['status'] = df_train['status'].map({'ham':0, 'spam':1})
df_test['status'] = df_test['status'].map({'ham':0, 'spam':1})
x_train = df_train['msg'].to_numpy()
y_train = df_train['status'].to_numpy()
x_test = df_test['msg'].to_numpy()
y_test = df_test['status'].to_numpy()


EPOCHS = 10

In [ ]:
words = []
max = 0
for msg in df_train['msg']:
    word_list = msg.lower().split()
    if len(word_list) > max:
        max = len(word_list)
    words.extend(word_list)

vocabulary_size = len(set(words))
total_word_count = len(words)
total_data = df_train.shape[0]
avg_word_per_msg = round(total_word_count / total_data)
max

In [ ]:
max_len = 100  # Maximum sequence length
tokenizer = Tokenizer(num_words=vocabulary_size, oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

# Convert text to sequences
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Pad sequences
x_train_padded = pad_sequences(x_train_seq, maxlen=max_len, padding='post', truncating='post')
x_test_padded = pad_sequences(x_test_seq, maxlen=max_len, padding='post', truncating='post')


In [ ]:
x_train_padded[0]


In [ ]:
# Build LSTM model
embedding_dim = 128

model = keras.Sequential([
    keras.layers.Embedding(vocabulary_size, embedding_dim, input_length=max_len),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()



In [ ]:
# Train the model
history = model.fit(
    x_train_padded,
    y_train,
    epochs=EPOCHS,
    validation_data=(x_test_padded, y_test),
    verbose=1
)

In [ ]:
loss, accuracy = model.evaluate(x_test_padded, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text_np = np.array([pred_text])
  pred_text_seq = tokenizer.texts_to_sequences(pred_text_np)
  pred_text_padded = pad_sequences(pred_text_seq, maxlen=max_len, padding='post', truncating='post')
  pred = model.predict(pred_text_padded, verbose=0)
  if pred[0][0] >= 0.5:
    prediction = [float(pred[0][0]) , 'spam']
  else:
    prediction = [float(pred[0][0]) , 'ham']
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
